# Installing Net2Brain

<img src="workshops/data/Net2Brain_Logo.png" width="25%" />

In [ ]:
#!pip install -U git+https://github.com/cvai-roig-lab/Net2Brain

# Step 3: Evaluation

In this tutorial notebook, we'll showcase how to leverage the evaluation capabilities of Net2Brain and visualize the resulting data. You can choose from three evaluation metrics:

1. "RSA"
2. "Weighted RSA"
3. "Searchlight"

Each module generates a pandas DataFrame, which can be seamlessly integrated with the toolbox's built-in plotting functionality.

### Representational Similiratiy Analysis (RSA) Evaluation

In [ ]:
from net2brain.evaluations.rsa import RSA
from net2brain.utils.download_datasets import DatasetBonnerPNAS2017
from pprint import pprint

First load the ROIs

In [ ]:
paths_bonner = DatasetBonnerPNAS2017.load_dataset()
stimuli_path = paths_bonner["stimuli_path"]
roi_path = paths_bonner["roi_path"]


In [ ]:
model_rdms = "AlexNet_RDM" 
brain_rdms = roi_path

# Start RSA
evaluation_alexnet = RSA(model_rdms, brain_rdms, model_name="AlexNet")

# Evaluation - Returns a pandas dataframe
dataframe1 = evaluation_alexnet.evaluate() 

# Show results
display(dataframe1)


The RSA class can also take an optional argument `layer_skips` to skip certain layers during evaluation. This can be
useful when you have a large number of layers extracted and want to test only a subset of them.

Also, by default the evaluation returns the squared correlations. If you don't want to square the correlations, you
can set `squared=False`. This is different from later changing the metric in plotting, as correlations are
subject-averaged by then, and also negative correlations cannot be recovered.

## Visualizing RSA Evaluation Results

The integrated plotting functionality of the toolbox allows you to easily visualize evaluation results. To achieve this, initialize the class with a list of DataFrames obtained from the evaluation. Make sure that each DataFrame:

1. Contains the same ROIs, signifying that each test was performed on the same brain RDMs.
2. Has a distinct model name, which can be set manually or through the "model_name" parameter during evaluation (as mentioned earlier).

Here's an example of how to plot the data using a single DataFrame:

In [ ]:
from net2brain.evaluations.plotting import Plotting

plotter = Plotting([dataframe1])
results_dataframe = plotter.plot()

You can choose between `metric="R2"` (default) or `metric="R"`

In [ ]:
results_dataframe = plotter.plot(metric="R")

You can also choose between plotting the best layers or `plot_all_layers()`.

In [ ]:
results_dataframe = plotter.plot_all_layers(metric="R2")

## Visualizing RSA Evaluation Results - Multiple models

As previously mentioned, you can also plot multiple models in a single plot. To do this, simply include additional DataFrames in the list:

        Plotting([dataframe1, dataframe2, dataframe3])



In [ ]:
# Start RSA for AlexNet
evaluation_alexnet = RSA("AlexNet_RDM", brain_rdms, save_path="./", model_name="AlexNet")
dataframe2 = evaluation_alexnet.evaluate() 

# Start RSA for ResNet50
evaluation_resnet = RSA("ResNet50_RDM", brain_rdms, save_path="./", model_name="ResNet50")
dataframe1 = evaluation_resnet.evaluate() 


plotter = Plotting([dataframe1,dataframe2])
results_dataframe = plotter.plot()

## Visualizing RSA Evaluation Results - Multiple models with significance


Furthermore, you might be interested in determining whether one model is significantly better than another, and not merely due to random variation. In this case, you can utilize the `compare_model` functionality provided by the toolbox. Use the following syntax:

        ttest, sig_pairs = eval_1.compare_model(eval_2)

If you wish to display the significance as well, use the parameter pairs=[].

In [ ]:
# Comparing statistical significance
ttest, sig_pairs = evaluation_alexnet.compare_model(evaluation_resnet)
print(sig_pairs)

# Plotting with significance
plotter = Plotting([dataframe1,dataframe2])
results_dataframe = plotter.plot(pairs=sig_pairs)

### WRSA Evaluation
In addition to the standard RSA, Net2Brain also supports weighted RSA (WRSA) as an evaluation metric. WRSA allows for the incorporation of weights into the analysis, providing an alternative approach to evaluating model performance and examining the relationship between neural representations and computational models.

In [ ]:
from net2brain.evaluations.weighted_rsa import WRSA

# Start RSA
evaluation = WRSA(model_rdms, brain_rdms, save_path="./", model_name="ResNet50")

# Evaluation - Returns a pandas dataframe
dataframe1 = evaluation.evaluate() 

display(dataframe1)

### Searchlight
The toolbox offers the capability to perform searchlight analysis using Searchlight RDMs in the [ROI, subject, stimuli, stimuli] format. Please note that this toolbox does not include RDMs for testing purposes. However, if you have access to RDMs, you are welcome to use this functionality to conduct searchlight analysis.

In [ ]:
from net2brain.evaluations.searchlight import Searchlight
model_rdms = "-"
searchlight_rdm = "-"

evaluation = Searchlight(model_rdms, searchlight_rdm, save_path="./")
evaluation.evaluate()

# Linear Encoding and Ridge Regression

The toolbox offers `Linear Encoding` and `Ridge Regression` as evaluation metrics. These methods use raw features, without converting them into Representational Dissimilarity Matrices (RDMs), to train an encoding model that predicts unseen brain data.

* **Linear Encoding** uses Principal Component Analysis (PCA) followed by Linear Regression to reduce the feature dimensionality and then predict brain activity. It helps understand which features from the model correlate with brain responses.

* **Ridge Regression** extends linear encoding by adding regularization, which prevents overfitting and helps dealing with noise which may be part of the neural data.

### `Encoding` Function Parameters:

- **feat_path (str)**: Path to the directory with model activation `.npz` files for multiple layers.
- **roi_path (str or list)**: Path to the directory containing .npy fMRI data files for multiple ROIs.
If we have a list of folders, each folder will be searched for .npy files and the analysis will be run for each. If 
folders contain different subject ROIs, make sure that the .npy file names are unique (e.g. V1_subj1.npy) across the 
folders.
- **model_name (str)**: Name of the model (used for labeling in output files).
- **trn_tst_split (int or float)**: Data to use for training (rest is used for testing). If int, it is absolute 
number of samples, if float, it is a fraction of the whole dataset (e.g., 0.8 means 80% training, 20% testing).
- **n_folds (int)**: Number of cross-validation folds.
- **n_components (int)**: Number of principal components to retain during PCA (only linear encoding)
- **batch_size (int)**: Batch size for Incremental PCA. (only linear encoding)
- **srp_before_pca (bool)**: Whether to apply Sparse Random Projection (SRP) before PCA. Use when features are so
high-dimensional that IncrementalPCA runs out of memory after some batches. Num of dims estimated by SRP. (only linear encoding)
- **srp_on_subset (int or None)**: Number of samples to use for SRP fitting. If None, all samples are used,
which is recommended if you have enough memory (if `srp_before_pca` is False it has no effect). (only linear encoding)
- **mem_mode (str)**: 'saver' or 'performance'; Choose 'saver' if you don't have enough memory to store all
training sample features, otherwise leave 'performance' as default. If you have `srp_before_pca` enabled,
in the first case you will also need to restrict the number of samples for SRP fitting with `srp_on_subset`. (only linear encoding)
- **avg_across_feat (bool)**: If True, averages activations across axis 1 to handle varying feature sizes (with LLMs for example).
- **return_correlations (bool)**: If True, return correlation values for each voxel (only with veRSA False).
- **random_state (int)**: Seed for reproducibility of results.
- **shuffle (bool)**: Whether to shuffle the data before splitting into training and testing sets.
- **save_path (str)**: Directory to save results (`Linear_Encoding_Results` by default).
- **file_name (str or None)**: Custom file name for saved results.
- **average_across_layers (bool)**: If True, averages correlation values across layers before saving the results.
- **veRSA (bool)**: If True, performs RSA on top of the voxelwise encoding.
- **save_model (bool)**: Save the linear regression model to disk.
- **save_pca (bool)**: Save the PCA transform to disk.
- **layer_skips (tuple, optional)**: Names of the model layers to skip during encoding. Use original layer names.


In [ ]:

# For a proper tutorial check out "notebooks/Workshops/Cognition Academy Dresden Notebook 2.ipynb"

from net2brain.evaluations.encoding import Linear_Encoding, Ridge_Encoding

Linear_Encoding(feat_path="feat_path",  # Or use Ridge Encoding
                roi_path="roi_path",
                model_name="model_name",
                save_path="save_path",
                file_name="file_name",
                avg_across_feat=True,
                return_correlations=False,
                average_across_layers=False)


Ridge_Encoding(feat_path="feat_path", 
                roi_path="roi_path",
                model_name="model_name",
                save_path="save_path",
                file_name="file_name",
                avg_across_feat=True,
                return_correlations=False,
                average_across_layers=False)

**For a proper tutorial check out "notebooks/Workshops/Cognition Academy Dresden Notebook 2.ipynb"**

### **Centered Kernel Alignment (CKA) Evaluation**

Centered Kernel Alignment (CKA) measures the similarity between two datasets, such as neural representations and brain activity, based on pairwise similarities within each dataset. It is scale-invariant and focuses on relationships rather than raw data.

#### **CKA Function Parameters**

- **`feat_path` (str):** Path to the directory containing `.npz` files with model activations for multiple layers. Each file should be organized by layer names.
- **`brain_path` (str):** Path to the directory containing `.npy` files with fMRI data for different ROIs. 
- **`model_name` (str):** Name of the model, used for labeling in the output.

#### **Input Format**

1. **Model Activations (`feat_path`):** Each `.npz` file corresponds to one stimulus and contains a dictionary of layer activations, just like the Feature Extractor prepares the files.
   - Example:
     ```python
     {
       'layer1': np.array([...]),  # Shape: (1, 64, 56, 56)
       'layer2': np.array([...]),  # Shape: (1, 128, 28, 28)
       ...
     }
     ```
2. **Brain Data (`brain_path`):** Each `.npy` file contains fMRI responses for an ROI, shaped as `(n_stimuli, n_voxels)`.


In [ ]:
import numpy as np
import os
from net2brain.evaluations.cka import CKA

# Define paths for dummy feature and brain data
feat_path = "dummy_feat_path"
brain_path = "dummy_brain_path"

# Ensure directories exist
os.makedirs(feat_path, exist_ok=True)
os.makedirs(brain_path, exist_ok=True)

# Number of stimuli and voxels
n_stimuli = 100
n_voxels = 10

# Generate dummy fMRI data
np.save(f"{brain_path}/roi1.npy", np.random.rand(n_stimuli, n_voxels))  # Shape: (10 stimuli, 1000 voxels)

# Generate dummy feature data for each stimulus
for i in range(n_stimuli):
    np.savez_compressed(f"{feat_path}/stimulus_{i+1}.npz",
                        layer1=np.random.rand(20),  # Example layer
                        layer2=np.random.rand(20))  # Example layer

# Run the CKA evaluation
results = CKA.run(feat_path=feat_path, brain_path=brain_path, model_name="clip_vit_b32")
display(results)


Otherwise, with raw data you can also call the computation directly!

In [ ]:
import numpy as np
import os
from net2brain.evaluations.cka import CKA

# Set a fixed random seed for reproducibility
np.random.seed(45)

# Generate dummy data
n_samples = 100                 # Number of samples (e.g., stimuli)
n_features_per_layer = 20       # Number of features in the DNN layer
n_voxels = 10                   # Number of fMRI response voxels

# Randomly generate DNN activations and fMRI responses
layer1_activations = np.random.rand(n_samples, n_features_per_layer)
fMRI_activations = np.random.rand(n_samples, n_voxels)

# Instantiate the CKA object
cka = CKA()

# Compute the linear CKA score between DNN activations and fMRI responses
cka_score = cka.linear_CKA(layer1_activations, fMRI_activations)

# Print the resulting CKA score
print("Linear CKA Score:", cka_score)

### **Distributional Comparison (DC) Evaluation**

Distributional Comparison evaluates the similarity between two datasets by comparing the distributions of their features. Two metrics are available:
- **Jensen-Shannon Divergence (JSD):** Measures the divergence between two probability distributions. It is symmetric and always bounded between 0 and 1.
- **Wasserstein Distance (WD):** Also known as Earth Mover's Distance, measures the cost of transforming one distribution into the other.

Use `metric="jsd"` for bounded divergence measures or `metric="wasserstein"` for more general comparisons.


#### **DC Function Parameters**

- **`feat_path` (str):** Path to the directory containing `.npz` files with model activations for multiple layers.
- **`brain_path` (str):** Path to the directory containing `.npy` files with fMRI data for different ROIs.
- **`metric` (str):** Metric to compare distributions ('jsd' or 'wasserstein').
- **`bins` (int):**  Number of bins for histogramming.
- **`model_name` (str):** Name of the model, used for labeling in the output.

#### **Input Format**

1. **Model Activations (`feat_path`):** Each `.npz` file corresponds to one stimulus and contains a dictionary of layer activations, just like the Feature Extractor prepares the files.
   - Example:
     ```python
     {
       'layer1': np.array([...]),  # Shape: (1, 64, 56, 56)
       'layer2': np.array([...]),  # Shape: (1, 128, 28, 28)
       ...
     }
     ```
2. **Brain Data (`brain_path`):** Each `.npy` file contains fMRI responses for an ROI, shaped as `(n_stimuli, n_voxels)`.

#### **Warning**

If the feature lengths of `feat_path` and `brain_path` differ, **PCA** is applied to reduce them to the same dimensionality. This step ensures compatibility but alters the feature space.


In [ ]:
import numpy as np
import os
from net2brain.evaluations.distributional_comparisons import DistributionalComparison


# Define paths for dummy feature and brain data
feat_path = "dummy_feat_path"
brain_path = "dummy_brain_path"

# Ensure directories exist
os.makedirs(feat_path, exist_ok=True)
os.makedirs(brain_path, exist_ok=True)

# Number of stimuli and voxels
n_stimuli = 100
n_voxels = 10

# Generate dummy fMRI data
np.save(f"{brain_path}/roi1.npy", np.random.rand(n_stimuli, n_voxels))  # Shape: (10 stimuli, 1000 voxels)

# Generate dummy feature data for each stimulus
for i in range(n_stimuli):
    np.savez_compressed(f"{feat_path}/stimulus_{i+1}.npz",
                        layer1=np.random.rand(20),  # Example layer
                        layer2=np.random.rand(20))  # Example layer

# Running Distributional Comparison (JSD)
results_jsd = DistributionalComparison.run(feat_path="dummy_feat_path", 
                                           brain_path="dummy_brain_path", 
                                           metric="jsd", 
                                           bins=50,
                                           model_name="clip_vit_b32")
print("Results (JSD):")
display(results_jsd)

# Running Distributional Comparison (Wasserstein)
results_wasserstein = DistributionalComparison.run(feat_path="dummy_feat_path", 
                                                   brain_path="dummy_brain_path", 
                                                   metric="wasserstein", 
                                                   bins=50,
                                                   model_name="clip_vit_b32")
print("Results (Wasserstein):")
display(results_wasserstein)


Otherwise, with raw data you can also call the computation directly!

In [ ]:
import numpy as np
import os
from net2brain.evaluations.distributional_comparisons import DistributionalComparison

# Set a fixed random seed for reproducibility
np.random.seed(45)

# Generate dummy data
n_samples = 100                 # Number of samples (e.g., stimuli)
n_features_per_layer = 20       # Number of features in the DNN layer
n_voxels = 20                   # Number of fMRI response voxels

# Randomly generate DNN activations and fMRI responses
layer1_activations = np.random.rand(n_samples, n_features_per_layer)
fMRI_activations = np.random.rand(n_samples, n_voxels)

# Instantiate the DC object
dc = DistributionalComparison()

# Compute the linear DC score between DNN activations and fMRI responses
dc_score = dc.compare_distributions(layer1_activations, fMRI_activations, metric="jsd")

# Print the resulting CKA score
print("DC Score:", dc_score)

# Stacked Encoding and Structured Variance Partitioning

## Stacked Encoding

Stacked encoding combines predictions from multiple feature spaces (like different neural network layers) to better predict brain activity. Instead of using just one layer or simply concatenating layers, stacked encoding:

### How Stacked Encoding Works:

1. **First-level models:** Train separate ridge regression models for each feature space (e.g., each layer of a neural network)
2. **Second-level combination:** Learn a convex combination of these individual predictions to create a final prediction

The weights of this combination indicate the importance of each feature space
These weights must be positive and sum to 1, ensuring interpretability


## Structured Variance Partitioning

Structured variance partitioning identifies which layers of a neural network are most important for predicting activity in different brain regions:

- **Forward direction**: Starts simple and adds complexity  
  - Identifies how complex representations need to be for prediction  

- **Backward direction**: Starts complex and adds simpler layers  
  - Determines if brain regions also process lower-level information  

Together, these analyses create an *interval* of relevant layers for each brain region. The `Stacked_Variance_Partitioning` function performs this analysis using R-Correlation values from stacked encoding.



### Code Example:

#### 1. Generate Random Data

In [ ]:
import numpy as np
import os

# Define parameters
N_sample = 1000  # Images
dim_X1 = 50  # Feature Dim 1
dim_X2 = 100  # Feature Dim 2
dim_X3 = 25  # Feature Dim 3
dim_Y = 10  # Amount Voxels

# Generate random feature data
X1 = np.random.randn(N_sample, dim_X1)
X2 = np.random.randn(N_sample, dim_X2)
X3 = np.random.randn(N_sample, dim_X3)

# Generate brain data
Y = 0.3 * X1.dot(np.random.randn(dim_X1, dim_Y)) + \
   0.3 * X2.dot(np.random.randn(dim_X2, dim_Y)) + \
   0.4 * X3.dot(np.random.randn(dim_X3, dim_Y))

# Define output directory
output_dir = "stacked_encoding_sample_data/features"
os.makedirs(output_dir, exist_ok=True)
os.makedirs("sample_data", exist_ok=True)

# Save each sample into a separate .npz file
for i in range(N_sample):
   file_path = os.path.join(output_dir, f"sample_{i+1:04d}.npz")
   np.savez(file_path, X1=X1[i], X2=X2[i], X3=X3[i])

# Save Y data as a single .npy file
y_file_path = "stacked_encoding_sample_data/brain_data.npy"
np.save(y_file_path, Y)

print(f"Data saved to {output_dir} with {N_sample} .npz files and Y.npy file of shape {Y.shape}.")


#### 2. Stacked Encoding + Stacked Variance Partitioning
(The Code automatically performs stacked VPA if vpa=True and saves the data locally)

In [ ]:
# Now perform stacked encoding analysis on the generated data
from net2brain.evaluations.stacked_encoding import Stacked_Encoding

# Execute stacked encoding with all parameters
results_df = Stacked_Encoding(
   feat_path="stacked_encoding_sample_data/features",       # Path to feature files (.npz files)
   roi_path="stacked_encoding_sample_data",                 # Path to brain data files
   model_name='SampleModel',                                # Name of the model for labeling
   n_folds=3,                                               # Number of cross-validation folds
   n_components=None,                                       # Number of PCA components (None to skip PCA)
   vpa=True,                                                # Whether to perform variance partitioning
   save_path='stacked_encoding_sample_data/results'         # Where to save results
)

results_df = results_df.drop(columns=["%R2", "LNC", "UNC"])
display(results_df)

#### 3. Stacked Variance Paritioning with my own data
In case you don't want to combine Stacked Encoding with VPA you can call the function with your own data as well:

In [ ]:
# If you want to run variance partitioning separately
from net2brain.evaluations.stacked_variance_partitioning import Stacked_Variance_Partitioning
import numpy as np

# In this case we are using the data we generated above!
r2s = np.load("stacked_encoding_sample_data/results/brain_data_SampleModel_stacked_encoding.npz", allow_pickle=True)["r2s"]
stacked_r2s = np.load("stacked_encoding_sample_data/results/brain_data_SampleModel_stacked_encoding.npz", allow_pickle=True)["stacked_r2s"]

# Run variance partitioning
vp_results = Stacked_Variance_Partitioning(
   r2s=r2s,                               # R² values for individual layers
   stacked_r2s=stacked_r2s,               # R² values for stacked model
   save_path='sample_data/vp_results'     # Where to save results
)

print("Forward layer assignments per voxel:", vp_results['vp_sel_layer_forward'])
print("Backward layer assignments per voxel:", vp_results['vpr_sel_layer_backward'])